In [31]:
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime
import arrow
import pathlib

In [32]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as pltz
import xarray as xar
import rioxarray

In [68]:
dir_to_read=r'F:\03nginx_data\nmefc_download\TY_GROUP_RESULT\TY2022_2021010416'
file_name='proSurge_TY2022_2021010416_gt0_5m.nc'


In [51]:
file_name.split('.')[0]

'proSurge_TY2022_2021010416_gt0_5m'

In [69]:
file_converted_name=file_name.split('.')[0]+'_converted.'+file_name.split('.')[1]
file_converted_name

'proSurge_TY2022_2021010416_gt0_5m_converted.nc'

In [70]:
# 尝试读取 pro 的nc文件
read_full_path=str(pathlib.Path(dir_to_read)/file_name)
read_full_path

'F:\\03nginx_data\\nmefc_download\\TY_GROUP_RESULT\\TY2022_2021010416\\proSurge_TY2022_2021010416_gt0_5m.nc'

In [71]:
ds_xr=xar.open_dataset(read_full_path)
ds_xr

<xarray.Dataset>
Dimensions:    (lat: 660, lon: 1080)
Dimensions without coordinates: lat, lon
Data variables:
    latitude   (lat) float32 ...
    longitude  (lon) float32 ...
    pro_surge  (lat, lon) float32 ...
Attributes:
    description:  The probability of storm surge >=0.5m

In [72]:
ds_xr=ds_xr.rename_vars({'latitude':'y','longitude':'x'})

In [73]:
ds_xr=ds_xr.swap_dims({'lat':'y','lon':'x'})

In [74]:
ds_xr.rio.set_spatial_dims("x","y",inplace=True)

<xarray.Dataset>
Dimensions:    (x: 1080, y: 660)
Coordinates:
  * y          (y) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
  * x          (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
Data variables:
    pro_surge  (y, x) float32 ...
Attributes:
    description:  The probability of storm surge >=0.5m

In [75]:
ds_xr=ds_xr.rio.write_crs("epsg:4326", inplace=True)

* 注意此处缺少 coords,设置了坐标系

ERROR: TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''  
ERROR2: DimensionError: x dimension not found. 'set_spatial_dims()' can address this.

注意使用 rioxarray -> geotiff 时，需要将 lon -> x, lat -> y

注意此处还需要加入一个步骤，对 y 也就是经度倒叙排列

In [60]:
ds_xr['y']=ds_xr['y'][::-1]

In [76]:
file_converted_full_path:str=str(pathlib.Path(dir_to_read)/file_converted_name)
file_converted_full_path

'F:\\03nginx_data\\nmefc_download\\TY_GROUP_RESULT\\TY2022_2021010416\\proSurge_TY2022_2021010416_gt0_5m_converted.nc'

In [77]:
ds_xr.to_netcdf(file_converted_full_path)

In [78]:
file_tif_name=file_converted_name.split('.')[0]+'.tif'
file_tif_full_path:str=str(pathlib.Path(dir_to_read)/file_tif_name)
file_tif_full_path

'F:\\03nginx_data\\nmefc_download\\TY_GROUP_RESULT\\TY2022_2021010416\\proSurge_TY2022_2021010416_gt0_5m_converted.tif'

In [79]:
# x dimension not found. 'set_spatial_dims()' can address this.
ds_xr.rio.to_raster(file_tif_full_path)

## 测试一下查看当前 `dataset` 的坐标系

In [65]:
ds_xr

<xarray.Dataset>
Dimensions:      (x: 1080, y: 660)
Coordinates:
  * y            (y) float32 25.991667 25.975 25.958334 ... 15.025 15.008333
  * x            (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
    spatial_ref  int32 0
Data variables:
    pro_surge    (y, x) float32 nan nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    description:   The probability of storm surge >=0.5m
    grid_mapping:  spatial_ref

In [67]:
ds_xr.pro_surge.rio.crs

CRS.from_epsg(4326)